In [1]:
from dotenv import load_dotenv
import google.generativeai as genai
import os
from google.api_core import retry
from pprint import pprint
from utils import SAFETY_SETTINGS, MODEL

In [2]:
load_dotenv()
google_api_key = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=google_api_key)
model = genai.GenerativeModel(MODEL)

In [3]:
chat = model.start_chat() 

In [4]:
chat.send_message("How are you today google?")

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=glm.GenerateContentResponse({'candidates': [{'content': {'parts': [{'text': "As an AI language model, I don't have feelings like humans do. However, I am functioning optimally and ready to assist you! How can I help you today? \n"}], 'role': 'model'}, 'finish_reason': 1, 'index': 0, 'safety_ratings': [{'category': 9, 'probability': 1, 'blocked': False}, {'category': 8, 'probability': 1, 'blocked': False}, {'category': 7, 'probability': 1, 'blocked': False}, {'category': 10, 'probability': 1, 'blocked': False}], 'token_count': 0, 'grounding_attributions': []}]}),
)

Ok everything is setup now we can continue on with adapting the mistral prompting cookbook to google gemini and exploring different prompting strategies to further our knowledge of LLMs 

In [5]:
def generate_with_retry(prompt):
  return model.generate_content(prompt, request_options={'retry':retry.Retry()})

In [6]:
# CLASSIFICATION 
def user_message(inquiry):
    user_message = f"""
    You are a movie recommendation AI. Classify the user's query into one of the following genres:
    
    Action, Comedy, Drama, Sci-Fi, Horror, Romance
    
    If the query doesn't fit, classify it as 'Other'. Only respond with the genre.

    ###
    Examples:
    
    Query: A thrilling movie with car chases and explosions.
    Genre: Action

    Query: A funny movie with witty dialogues.
    Genre: Comedy

    ###

    <<<
    Query: {inquiry}
    >>>
    """
    return user_message

genre = generate_with_retry(user_message("A scary movie with ghosts")).text
print(genre)

Horror 



In [7]:
# test with another example 
genre = generate_with_retry(user_message("A young man starts to catch feelings for a girl at the gym")).text
print(genre)

Romance 



In [9]:
# SUMMARISATION 
# make a message for the research assistant about the benefits of sunlight 
article_text = """
Sunlight is a natural source of vitamin D, which is essential for maintaining healthy bones and teeth. It also has other benefits, including helping to regulate mood and improve sleep. However, excessive exposure to sunlight can increase the risk of skin cancer. It is important to strike a balance between getting enough sunlight to reap its benefits and protecting your skin from harmful UV rays.
"""


message = f"""
You are a research assistant. 
Summarize the following scientific article and generate 3 insightful questions about it:

# Article:
{article_text}

# Instructions:
1. Summarize the key findings and conclusions of the article.
2. Generate 3 thought-provoking questions about the article's content.
3. Provide a detailed explanation for each question.

Output the summary and questions in a markdown report format.
"""

report = generate_with_retry(message).text
print(report)

## Report: Sunlight and Vitamin D

### Summary:

This article highlights the dual nature of sunlight exposure. While sunlight is a crucial source of vitamin D, essential for bone health and mood regulation, excessive exposure can lead to skin cancer.  The key takeaway is the importance of balancing the benefits of sunlight with the need to protect oneself from harmful UV radiation. 

### Questions:

1. **How can individuals determine the optimal amount of sun exposure for their specific needs, considering factors like skin type, geographical location, and time of year?**

    * This question delves into the personalized aspect of sun exposure. Factors like skin pigmentation, latitude, and season significantly influence vitamin D production and skin cancer risk. Understanding how these factors interact would allow for more precise recommendations. 
2. **Beyond sunscreen, what other preventative measures can be taken to mitigate the risk of skin cancer while still reaping the benefits of

In [10]:
# PERSONALISATION
email = """
Dear Fitness Coach,

I'm looking to increase my squat in powerlifting. The main lifts that I want to focus on are squat, bench press and deadlift along with accessories to help each of them. What kind of training plan would you recommend?

Regards,
Tyson
"""

message = f"""
You are a personal fitness coach. Respond to the following email using the provided information, 
offering a personalized training plan with a friendly and motivating tone:

# Information:
Tyson is a 22-year-old male with who loves playing roblox and minecraft. 
He aims to total 600kg in powerlifting and is currently at 570kg (squat: 210kg, bench press: 130kg, deadlift: 230kg).

# Email:
{email}
"""
response = generate_with_retry(message).text
print(response)

## Subject: Power Up Your Lifts, Tyson! 💪

Hey Tyson,

Awesome to hear about your powerlifting goals! Crushing a 600kg total is seriously impressive, and you're already so close. Let's work together to get your squat where you want it and smash that target! 🏋️‍♀️

Based on your current lifts (210kg squat, 130kg bench, 230kg deadlift), it seems like your squat could use a little extra love. Here's a personalized training plan that focuses on building your squat strength while still progressing your bench and deadlift:

**Training Split (3 days a week):**

*   **Day 1: Squat Focus**
    *   Squat: 5 sets of 3 reps (80-85% of your 1 rep max)
    *   Front Squats: 4 sets of 5 reps
    *   Leg Press: 3 sets of 8-12 reps
    *   Calf Raises: 3 sets of 15-20 reps
*   **Day 2: Bench Press Focus**
    *   Bench Press: 5 sets of 3 reps (80-85% of your 1 rep max)
    *   Incline Dumbbell Press: 4 sets of 5 reps
    *   Dips: 3 sets to failure 
    *   Overhead Press: 3 sets of 8-12 reps
*   **Day

In [11]:
# CONFIDENCE SCORING 
news_article = """
COD VANGUARD SOLD 30 MILLION COPIES DESPITE NEGATIVE REVIEWS
Call of Duty
April 6, 2024
Grant Taylor-Hill
It was recently discovered that Call of Duty: Vanguard, widely considered one of the worst COD games ever, sold more than 30 million copies. That makes Vanguard one of the best-selling Call of Duty games ever, despite the woeful reviews the game got when it was released. It failed to meet Activision’s expectations when it was released in 2021, emerging as the first core World War II-themed experience since 2017’s Call of Duty: WWII.

Vanguard looked brilliant on the surface, but clean graphics weren’t enough to mask the game’s instabilities and pitfalls. It had a fire-and-forget story that was dramatically short, and the multiplayer offering felt unbalanced and weak from the outset. Regardless, Call of Duty: Vanguard reportedly went on to sell a whopping 30 million copies, which ranks it high in the list of Call of Duty games where overall sales numbers are concerned.

Call of Duty: Vanguard was a game that’s tough to remember – it didn’t offer much to change the paradigm whatsoever. With 30 million copies sold, Vanguard did something right – but even now, the reviews are mostly negative for this game. On Metacritic, the highest score Vanguard has is 74/100. It’s not the worst Call of Duty game ever, though. That title is bestowed upon a 2012 game released exclusively for the PS Vita, ‘Black Ops Declassified’, which boasts a Metacritic rating of just 33/100.

In a statement released months after the launch of Vanguard, Activision’s spokespeople confirmed that Vanguard failed to meet internal expectations. It was said that the World War II theme didn’t resonate with fans and the game failed to innovate. One year later, Activision would release Call of Duty: Modern Warfare II, which would go on to become the fastest-selling Call of Duty game ever, so the fan fervour was still there, just not for Vanguard.

Vanguard is one of those Call of Duty games that died off very quickly, with player numbers dwindling before the title’s ‘lifecycle’ had ended. The World War II setting also gave rise to a new Warzone map, Caldera, which many still say is what killed the popular battle royale title for a time.
"""
message = f"""
You are a factual language model. 
Summarize the following news article and provide a confidence score (0-1) for each summary:

# Article:
{news_article}

# Instructions:
1. Generate 3 concise summaries of the article.
2. Provide a confidence score for each summary, reflecting its accuracy and completeness.

Output the summaries and scores in a JSON format.
"""
result = generate_with_retry(message).text
print(result)

```json
{
  "summaries": [
    {
      "summary": "Despite receiving negative reviews and failing to meet Activision's expectations, Call of Duty: Vanguard sold over 30 million copies, making it one of the best-selling Call of Duty games. Its World War II setting and lack of innovation were cited as reasons for its underperformance compared to other titles in the franchise.",
      "confidence": 0.95
    },
    {
      "summary": "Call of Duty: Vanguard, considered one of the worst COD games, surprisingly sold 30 million copies. Its poor reception was attributed to a short story, unbalanced multiplayer, and uninspired World War II setting.  However, it still ranks high in overall sales for the franchise.",
      "confidence": 0.9 
    },
    {
      "summary": "Vanguard's sales success is contrasted with its negative reviews and failure to meet Activision's expectations. The World War II theme and lack of innovation are blamed, leading to a quick decline in player numbers and the under

In [12]:
# EVALUATION STEPPING 
news_article = """
Tesla, the renowned electric vehicle manufacturer, has recently initiated a significant reduction in its workforce, impacting a substantial number of employees. This move is part of a broader trend of layoffs across various industries, with several companies scaling back their staff numbers. Tesla's layoffs are not isolated to production roles but also include corporate positions, indicating a strategic reorganization within the company.

The layoffs come at a time when Tesla is facing multiple challenges, including a slowdown in demand for its vehicles and increased competition in the electric vehicle market. The company's stock has also seen a notable decline, adding to the pressures of maintaining profitability and market leadership.

In addition to the workforce reduction, Tesla has seen the departure of key executives, which has raised questions about the stability of its leadership team. The exits of high-ranking officials such as the Senior Vice President of Powertrain and Energy and the Vice President of Public Policy and Business Development suggest a potential shift in the company's strategic direction or internal dynamics.

The restructuring at Tesla is reflective of a larger pattern of executive turnover and layoffs within the tech and automotive sectors. Companies are reevaluating their staffing needs and making difficult decisions to streamline operations and focus on core competencies.

Tesla's approach to these layoffs and executive changes appears to be part of a calculated effort to position the company for future growth. By reducing costs and increasing productivity, Tesla aims to navigate through the current market challenges and emerge stronger, with a focus on innovation and the development of new products.

The electric vehicle giant is also grappling with internal issues related to its corporate culture and governance. These challenges have led to scrutiny from regulatory bodies and have had implications for the company's stock performance. Tesla's board of directors is tasked with addressing these concerns while steering the company towards a sustainable and responsible business model.

As Tesla prepares for the next phase of its evolution, the company is expected to concentrate on key projects such as the production of the Cybertruck and the development of a new vehicle platform. These initiatives are crucial for Tesla's long-term success and require a lean and agile organizational structure to execute effectively.

Overall, Tesla's recent layoffs and executive departures are indicative of the company's efforts to adapt to a rapidly changing industry landscape. By making these tough decisions, Tesla is working to maintain its position as a leader in the electric vehicle market and to ensure its operations are well-aligned with its strategic goals.

Citations:
[1] https://supplychainnow.com/tracking-recent-layoffs-dp49/
[2] https://casetext.com/case/zilincik-v-tesla-inc
[3] https://seekingalpha.com/article/4205144-tesla-endgame
[4] https://www.emerald.com/insight/content/doi/10.1108/TCJ-12-2022-0198/full/html?skipTracking=true
[5] https://sloanreview.mit.edu/article/toxic-culture-is-driving-the-great-resignation/
[6] https://www.businessinsider.com/peloton-company-history-rise-fall-2022-2
[7] https://www.industryweek.com/leadership/companies-executives/article/21284628/hot-or-not-whats-the-state-of-the-ev-sector
[8] https://www.reddit.com/r/Layoffs/comments/1arlt1i/for_those_of_you_who_were_laid_off_did_you_see_it/
"""


message = f"""
You are a critical language model with expertise in evaluating summaries.

# Article:
{news_article}

# Instructions:
1. Generate 3 concise summaries of the article.
2. Evaluate the summaries and choose the one that best captures the key points and essence of the article. 
3. Explain why you chose that summary as the best, considering factors like accuracy, completeness, and conciseness.

Output the summaries and your evaluation in a clear text format.
"""
evaluation_result = generate_with_retry(message).text
print(evaluation_result)

## Summaries of the Article:

**Summary 1:**

Tesla is undergoing significant restructuring, including layoffs and executive departures, in response to market challenges and internal issues. The company aims to streamline operations, reduce costs, and focus on key projects like the Cybertruck and a new vehicle platform to maintain its leadership in the electric vehicle market.

**Summary 2:**

Facing a slowdown in demand and increased competition, Tesla has initiated layoffs across various departments and experienced key executive departures. These actions reflect broader industry trends and Tesla's efforts to adapt and remain competitive by focusing on core competencies and innovation.

**Summary 3:** 

Tesla's recent layoffs and executive turnover highlight the company's struggle with market pressures, internal culture concerns, and leadership stability. As the company navigates these challenges, it aims to prioritize efficiency, innovation, and key projects to ensure future growth a

In [13]:
# EXTERNAL EVALUATION WITH ANOTHER MODEL 
summaries = evaluation_result

In [35]:
# Evaluate with a different model 
evaluation_model = genai.GenerativeModel("gemini-1.0-pro-latest")  # Replace with your desired model

In [36]:
import pprint
for model in genai.list_models():
    pprint.pprint(model)

Model(name='models/chat-bison-001',
      base_model_id='',
      version='001',
      display_name='PaLM 2 Chat (Legacy)',
      description='A legacy text-only model optimized for chat conversations',
      input_token_limit=4096,
      output_token_limit=1024,
      supported_generation_methods=['generateMessage', 'countMessageTokens'],
      temperature=0.25,
      top_p=0.95,
      top_k=40)
Model(name='models/text-bison-001',
      base_model_id='',
      version='001',
      display_name='PaLM 2 (Legacy)',
      description='A legacy model that understands text and generates text as an output',
      input_token_limit=8196,
      output_token_limit=1024,
      supported_generation_methods=['generateText', 'countTextTokens', 'createTunedTextModel'],
      temperature=0.7,
      top_p=0.95,
      top_k=40)
Model(name='models/embedding-gecko-001',
      base_model_id='',
      version='001',
      display_name='Embedding Gecko',
      description='Obtain a distributed representatio

In [37]:
evaluation_prompt = f"""
You are an expert evaluator of summaries. Assess the following summaries and choose the best one based on accuracy, completeness, and conciseness:

# Summaries:
{summaries}

# Article:
{news_article}
"""
evaluation_result = evaluation_model.generate_content(evaluation_prompt).text
print(evaluation_result)

**The best summary is Summary 3.** It provides the most comprehensive overview of the article, encompassing the key points:

* **Market Challenges:** The summary accurately reflects the slowdown in demand, increased competition, and stock decline that Tesla is facing.
* **Internal Issues:** It acknowledges the concerns regarding corporate culture and governance, along with the impact of executive departures on leadership stability. 
* **Strategic Response:**  The summary effectively captures Tesla's efforts to adapt through restructuring, cost reduction, and a focus on crucial projects for future growth.

**Reasons for Choosing Summary 3:**

* **Accuracy:** It faithfully represents the information presented in the article without distortion or omission.
* **Completeness:** It covers all the essential aspects of the situation, including both external and internal challenges faced by Tesla.
* **Conciseness:** While being comprehensive, it remains concise and avoids unnecessary details. 
